In [1]:
!pip install --quiet --upgrade google-cloud-logging google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently t

In [1]:
import vertexai
import logging
import google.cloud.logging
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

import pickle
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

In [4]:
# Install the Vertex AI Python SDK if not already installed
!pip install --upgrade google-cloud-aiplatform

# Import the Vertex AI SDK
from google.cloud import aiplatform

# Initialize Vertex AI with your project and location
aiplatform.init(
    project="qwiklabs-gcp-02-d8cd54229bbe",
    location="us-central1"
)

print("Vertex AI initialized successfully.")


Vertex AI initialized successfully.


In [5]:
from langchain_google_vertexai import VertexAIEmbeddings

# Create the embedding model instance
embedding_model = VertexAIEmbeddings(
    model_name="text-embedding-005"
)


/usr/local/lib/python3.11/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [6]:
!gcloud storage cp gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf .

Copying gs://partner-genai-bucket/genai069/nyc_food_safety_manual.pdf to file://./nyc_food_safety_manual.pdf

Average throughput: 162.9MiB/s


In [9]:
from google.cloud import storage
from langchain_community.document_loaders import PyMuPDFLoader

# GCS details
bucket_name = "partner-genai-bucket"
blob_name = "genai069/nyc_food_safety_manual.pdf"
local_path = "nyc_food_safety_manual.pdf"

# Download from GCS
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.download_to_filename(local_path)

# Load PDF locally
loader = PyMuPDFLoader(local_path)
data = loader.load()
print(f"Loaded {len(data)} pages")


Loaded 94 pages


In [10]:
def clean_page(page):
  return page.page_content.replace("-\n","")\
                          .replace("\n"," ")\
                          .replace("\x02","")\
                          .replace("\x03","")\
                          .replace("fo d P R O T E C T I O N  T R A I N I N G  M A N U A L","")\
                          .replace("N E W  Y O R K  C I T Y  D E P A R T M E N T  O F  H E A L T H  &  M E N T A L  H Y G I E N E","")

In [12]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

# 1. Load the PDF
loader = PyMuPDFLoader("nyc_food_safety_manual.pdf")
pages = loader.load()

# 2. Define your cleaning function
def clean_page(page):
    return page.page_content.replace("-\n", "") \
                            .replace("\n", " ") \
                            .replace("\x02", "") \
                            .replace("\x03", "") \
                            .replace("fo d P R O T E C T I O N  T R A I N I N G  M A N U A L", "") \
                            .replace("N E W  Y O R K  C I T Y  D E P A R T M E N T  O F  H E A L T H  &  M E N T A L  H Y G I E N E", "")

# 3. Apply cleaning to each page
cleaned_pages = []
for page in pages:
    page.page_content = clean_page(page)
    cleaned_pages.append(page)

# 4. Create the SemanticChunker with your embedding model
chunker = SemanticChunker(embedding_model)

# 5. Take only the first 5 cleaned pages
first_five_pages = cleaned_pages[:5]

# 6. Split into semantic chunks
chunked_docs = chunker.split_documents(first_five_pages)

# 7. Extract just the text content
chunked_content = [doc.page_content for doc in chunked_docs]

# 8. Preview a few chunks
for i, chunk in enumerate(chunked_content[:5], start=1):
    print(f"--- Chunk {i} ---")
    print(chunk)
    print()


--- Chunk 1 ---
The Health Code These are regulations that were formulated to allow the  Department to effectively protect the health of the population. Among the rules embodied in the Health Code is Article 81 which regulates the operations of food establishments for the purpose of preventing public health hazards. Environmental Health Division  The Division of Environmental Health is the Commission within the Health Department that is concerned with public health and works to eliminate the incidence of injury and illness caused by environmental factors. There are several Offices and Bureaus within this division. One of these is the Bureau of Food Safety and Community Sanitation that has the responsibility for conducting inspections of food service and food processing establishments. These inspections are performed by Public Health Sanitarians. Anti-corruption Warning All Sanitarians have Department of Health and Mental Hygiene badges and identification cards which they must display w

In [13]:
# Generate embeddings for the text chunks
chunked_embeddings = embedding_model.embed_documents(chunked_content)

# Check how many embeddings were created and preview the first one
print(f"Generated {len(chunked_embeddings)} embeddings.")
print("First embedding vector (truncated):", chunked_embeddings[0][:10])


Generated 15 embeddings.
First embedding vector (truncated): [-0.04205901920795441, -0.018308516591787338, -0.01328700315207243, -0.05176236853003502, 0.016785096377134323, 0.034466180950403214, 0.013948620297014713, 0.02817848138511181, 0.02424997091293335, 0.009253749623894691]


In [20]:
import pickle

from google.cloud import logging as cloud_logging
import logging

log_message = f"chunked contents are: {chunked_content[0][:20]}"
logging.info(log_message)

!gcloud storage cp gs://partner-genai-bucket/genai069/chunked_content.pkl .
!gcloud storage cp gs://partner-genai-bucket/genai069/chunked_embeddings.pkl .

chunked_content = pickle.load(open("chunked_content.pkl", "rb"))
chunked_embeddings = pickle.load(open("chunked_embeddings.pkl", "rb"))

# Do not delete this logging statement.
client = cloud_logging.Client()
client.setup_logging()

log_message = f"chunked contents are: {chunked_content[0][:20]}"
logging.info(log_message)

INFO:root:chunked contents are: The Health Code Thes


Copying gs://partner-genai-bucket/genai069/chunked_content.pkl to file://./chunked_content.pkl
Copying gs://partner-genai-bucket/genai069/chunked_embeddings.pkl to file://./chunked_embeddings.pkl

Average throughput: 150.4MiB/s


INFO:root:chunked contents are: The Health Code Thes


In [21]:
from google.cloud import firestore

# Initialize the Firestore client
db = firestore.Client()

print("Firestore client initialized for project:", db.project)


Firestore client initialized for project: qwiklabs-gcp-02-d8cd54229bbe


In [22]:
collection = db.collection("food-safety")

In [23]:
from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector  # Firestore Vector type

# Initialize Firestore client (db should already be set, but just in case)
db = firestore.Client()

# Name of your collection
collection_name = "food-safety"

# Loop through chunks and embeddings together
for content, embedding in zip(chunked_content, chunked_embeddings):
    doc_ref = db.collection(collection_name).document()  # random ID
    doc_ref.set({
        "content": content,
        "embedding": Vector(embedding)
    })

print(f"Inserted {len(chunked_content)} documents into '{collection_name}' collection.")


Inserted 273 documents into 'food-safety' collection.


In [34]:
!gcloud firestore indexes composite create \
--project="qwiklabs-gcp-02-d8cd54229bbe" \
--collection-group="food-safety" \
--query-scope=COLLECTION \
--field-config=vector-config='{"dimension":"768","flat": "{}"}',field-path=embedding


Create request issued
Created index [CICAgOjXh4EK].


In [37]:
def search_vector_database(query: str):
    query_embedding = embedding_model.embed_query(query)
    query_vector = Vector(query_embedding)
    docs = collection.find_nearest(
        "embedding",
        query_vector=query_vector,
        distance_measure=DistanceMeasure.DOT_PRODUCT,
        limit=5
    ).get()

    pieces = []
    for doc in docs:
        data = doc.to_dict()
        if "content" in data:
            pieces.append(data["content"])

    context = "\n".join(pieces)
    return context

# Test the function
result = search_vector_database("How should I store food?")
print(result)

 Store foods away from dripping condensate , at least six inches above the floor and with enough space between items to encourage air circulation. Freezer Storage Freezing is an excellent method for prolonging the shelf life of foods. By keeping foods frozen solid, the bacterial growth is minimal at best. However, if frozen foods are thawed and then refrozen, then harmful bacteria can reproduce to dangerous levels when thawed for the second time. In addition to that, the quality of the food is also affected. Never refreeze thawed foods, instead use them immediately. Keep the following rules in mind for freezer storage:  Use First In First Out method of stock rotation. All frozen foods should be frozen solid with temperature at 0°F or lower. Always use clean containers that are clearly labeled and marked, and have proper and secure lids. Allow adequate spacing between food containers to allow for proper air circulation. Never use the freezer for cooling hot foods. * * Tip: When receivin

In [39]:
search_vector_database("How should I store food?")

' Store foods away from dripping condensate , at least six inches above the floor and with enough space between items to encourage air circulation. Freezer Storage Freezing is an excellent method for prolonging the shelf life of foods. By keeping foods frozen solid, the bacterial growth is minimal at best. However, if frozen foods are thawed and then refrozen, then harmful bacteria can reproduce to dangerous levels when thawed for the second time. In addition to that, the quality of the food is also affected. Never refreeze thawed foods, instead use them immediately. Keep the following rules in mind for freezer storage:  Use First In First Out method of stock rotation. All frozen foods should be frozen solid with temperature at 0°F or lower. Always use clean containers that are clearly labeled and marked, and have proper and secure lids. Allow adequate spacing between food containers to allow for proper air circulation. Never use the freezer for cooling hot foods. * * Tip: When receivi